In [1]:
%load_ext dotenv
%dotenv .env

In [ ]:
from redbox.app import Redbox
from redbox.models.settings import Settings, get_settings
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, ChatLLMBackend
from langfuse.callback import CallbackHandler
from uuid import uuid4
from langchain_core.runnables.graph import MermaidDrawMethod
import langchain

In [3]:
def get_state(user_uuid, prompts, documents, ai_setting):
    q = RedboxQuery(
        question=f"@search {prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )

    return RedboxState(
        request=q,
    )

In [5]:
def run_app(app, state) -> RedboxState:
    langfuse_handler = CallbackHandler()
    return app.graph.invoke(state, config={"callbacks": [langfuse_handler]})

env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ['natasha.boyse@digital.trade.gov.uk/179._Design_Council.pdf']
user_prompt = "How does the exposure to AI vary across different geographical areas?"

x = get_state(uuid4(), prompts = [user_prompt], documents = documents, ai_setting = ai_setting)
result = run_app(app, x)

In [ ]:
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
documents = ['natasha.boyse@digital.trade.gov.uk/179._Design_Council.pdf']
user_prompt = "How does the exposure to AI vary across different geographical areas?"

llm_rephrased = []
final_answer = []
for i in range(5):
    x = get_state(uuid4(), prompts = [user_prompt], documents = documents, ai_setting = ai_setting)
    result = run_app(app, x)
    llm_rephrased.append(result["messages"][0].content)
    final_answer.append(result["messages"][1].content)

In [ ]:
llm_rephrased[0], final_answer[0]

In [ ]:
llm_rephrased[1], final_answer[1]

In [ ]:
llm_rephrased[2], final_answer[2]

In [ ]:
llm_rephrased[3], final_answer[3]

In [ ]:
print(result["messages"][1].content)

In [ ]:
graph = app.graph.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")

In [ ]:
graph = app.graph.get_graph().nodes['p_retrieve_metadata'].data.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")

In [ ]:
graph = app.graph.get_graph().nodes['p_search'].data.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")

In [ ]:
graph = graph.nodes['s_tool'].data.get_graph()
graph.draw_mermaid_png(draw_method=MermaidDrawMethod.API, output_file_path="./graph.png")